<a href="https://colab.research.google.com/github/prady1900/Kaggle_Competition/blob/main/Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload() #upload kaggle.json

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

!kaggle kernels list — user prady1900 — sort-by dateRun

!kaggle competitions download -c tabular-playground-series-feb-2021

!unzip -q train.csv.zip -d .
!unzip -q test.csv.zip -d .
!ls

Saving kaggle.json to kaggle.json
kaggle.json
usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: unrecognized arguments: — user prady1900 — sort-by dateRun
  0% 0.00/470k [00:00<?, ?B/s]
100% 470k/470k [00:00<00:00, 32.1MB/s]
 96% 24.0M/25.1M [00:00<00:00, 19.8MB/s]
100% 25.1M/25.1M [00:01<00:00, 16.8MB/s]
 82% 33.0M/40.2M [00:00<00:00, 23.9MB/s]
100% 40.2M/40.2M [00:01<00:00, 39.5MB/s]
kaggle.json  sample_submission.csv.zip	test.csv.zip  train.csv.zip
sample_data  test.csv			train.csv


# New Section

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df =pd.read_csv('train.csv')
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,A,B,A,A,B,D,A,E,C,I,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,B,A,A,A,B,B,A,E,A,F,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,A,A,A,C,B,D,A,B,C,N,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,A,A,A,C,B,D,A,E,G,K,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,A,B,A,A,B,B,A,E,C,F,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [ ]:
df['target'].unique()

array([6.99402255, 8.07125606, 5.76045565, ..., 7.60055795, 8.27209531,
       6.02568526])

In [ ]:
df['target'].nunique()

299648

In [ ]:
df.shape

(300000, 26)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
cat_cols = df.iloc[:,1:11]
cat_cols[:5]

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9
0,A,B,A,A,B,D,A,E,C,I
1,B,A,A,A,B,B,A,E,A,F
2,A,A,A,C,B,D,A,B,C,N
3,A,A,A,C,B,D,A,E,G,K
4,A,B,A,A,B,B,A,E,C,F


In [ ]:
lbl = LabelEncoder()
for cols in cat_cols.columns:
  lbl.fit(df[cols])
  df[cols] = lbl.transform(df[cols])
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,0,1,0,0,1,3,0,4,2,8,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,1,0,0,0,1,1,0,4,0,5,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,0,0,0,2,1,3,0,1,2,13,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,0,0,0,2,1,3,0,4,6,10,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,0,1,0,0,1,1,0,4,2,5,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
for cols in cat_cols:
  df[cols] = scaler.fit_transform(df[[cols]])
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,0.0,1.0,0.0,0.000000,0.333333,1.000000,0.0,0.571429,0.333333,0.571429,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,1.0,0.0,0.0,0.000000,0.333333,0.333333,0.0,0.571429,0.000000,0.357143,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,0.0,0.0,0.0,0.666667,0.333333,1.000000,0.0,0.142857,0.333333,0.928571,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,0.0,0.0,0.0,0.666667,0.333333,1.000000,0.0,0.571429,1.000000,0.714286,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,0.0,1.0,0.0,0.000000,0.333333,0.333333,0.0,0.571429,0.333333,0.357143,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca=PCA(n_components=2)

In [ ]:
pca.fit(df.iloc[:,:-1])

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [ ]:
x_pca=pca.transform(df.iloc[:,:-1])

In [ ]:
y=df.iloc[:,-1]
y.head()

0    6.994023
1    8.071256
2    5.760456
3    7.806457
4    6.868974
Name: target, dtype: float64

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge=Ridge()
parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
ridge_regressor.fit(x_pca,y)

GridSearchCV(cv=5, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20, 30, 35, 40, 45, 50, 55, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
lasso=Lasso()
parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
lasso_regressor=GridSearchCV(lasso,parameters,scoring='r2_score',cv=5)

lasso_regressor.fit(x_pca,y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92619.54126903306, tolerance: 18.915062024880523
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92781.41249360447, tolerance: 18.890087868013815
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92536.66597446598, tolerance: 18.884295023384173
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92827.053

GridSearchCV(cv=5, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20, 30, 35, 40, 45, 50, 55, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

{'alpha': 1}
-0.7550882618565516


In [ ]:
df2 =  pd.read_csv('test.csv')

In [ ]:
cat_cols1 = df2.iloc[:,1:11]
for cols in cat_cols1.columns:
  lbl.fit(df2[cols])
  df2[cols] = lbl.transform(df2[cols])
df2.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,0,1,0,2,1,3,0,4,4,6,0.331261,0.551131,0.486284,0.156058,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448
1,5,0,1,0,2,1,3,0,4,2,11,0.402404,0.617546,0.296852,0.187440,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251
2,15,0,1,0,2,1,3,0,4,2,5,0.530677,0.618126,0.760865,0.199872,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905
3,16,0,0,1,0,1,3,0,4,4,5,0.439653,0.487403,0.715786,0.192808,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625
4,17,0,1,0,0,1,1,0,4,4,8,0.389844,0.752760,0.315269,0.191734,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931


In [ ]:
for cols in cat_cols1:
  df2[cols] = scaler.transform(df2[[cols]])
df2.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,0.0,0.071429,0.000000,0.142857,0.071429,0.214286,0.0,0.285714,0.285714,0.428571,0.331261,0.551131,0.486284,0.156058,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448
1,5,0.0,0.071429,0.000000,0.142857,0.071429,0.214286,0.0,0.285714,0.142857,0.785714,0.402404,0.617546,0.296852,0.187440,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251
2,15,0.0,0.071429,0.000000,0.142857,0.071429,0.214286,0.0,0.285714,0.142857,0.357143,0.530677,0.618126,0.760865,0.199872,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905
3,16,0.0,0.000000,0.071429,0.000000,0.071429,0.214286,0.0,0.285714,0.285714,0.357143,0.439653,0.487403,0.715786,0.192808,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625
4,17,0.0,0.071429,0.000000,0.000000,0.071429,0.071429,0.0,0.285714,0.285714,0.571429,0.389844,0.752760,0.315269,0.191734,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931


In [ ]:
x_test_pca=pca.transform(df2)

In [ ]:
y_pred = ridge_regressor.predict(x_test_pca)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
sub = pd.DataFrame()

In [ ]:
sub["id"] = df2["id"]
sub["target"] = y_pred

In [ ]:
sub.head()

,id,target
0,0,7.485814
1,5,7.447210
2,15,7.438921
3,16,7.459336
4,17,7.496144


In [ ]:
sub.to_csv('submission.csv', index=False)